In [18]:
import ML_Coupling.tools as tool
import numpy as np
import subprocess, time

xyz_path = './../ML_coupling/input/nat_dimer_para_x0_y0_z35.xyz'
n_atom, atom_sym, geo = tool.read_xyz(xyz_path, output='regular')
n_atom_frag1 = 18
n_atom_frag2 = n_atom - n_atom_frag1
geo_frag1 = geo[0 : n_atom_frag1]
geo_frag2 = geo[n_atom_frag1 : n_atom]

# shift frag1
geo_frag1_ = np.zeros(geo_frag1.shape)
geo_ = np.zeros(geo.shape)

# specify the scanning range
x_shift = np.arange(0, 4.1, 0.1)
y_shift = np.arange(0, 4.1, 0.1)
z_shift = np.arange(5.9, 6.1, 0.1)
x_rot = np.arange(0.0, 91, 2)
y_rot = np.arange(0.0, 91, 2)
z_rot = np.arange(0.0, 91, 2)

# qchem_input_template = '/home/jingheng/ML_data_set/QC_coupling/raw/nat_dimer/template_input.inp'
# with open(qchem_input_template, 'r') as f:
#     qc_input = f.readlines()
t0 = time.time()
check = tool.check_close_mol

trans_rot = []
idx = 0
for k in z_shift:
    k_ = k * 10
    geo_frag1_[:,2] = geo_frag1[:,2] + k
    for i in x_shift:
        i_ = i * 10
        geo_frag1_[:,0] = geo_frag1[:,0] + i
        # input_path = 'z%02d_x%02d'%(int(k_), int(i_))
#        subprocess.run('mkdir '+ input_path, shell=True)
        for j in y_shift:
            j_ = j * 10
            geo_frag1_[:,1] = geo_frag1[:,1] + j            
            # file_name = '/nat_dimer_para_xrot%03d_yrot%03d_zrot%03d.inp'%(int(i), int(j), int(k))

            for kr in z_rot:
                kr_ = kr / 180 * np.pi
                z_rot_tm = np.array([[np.cos(kr_), -np.sin(kr_), 0],[np.sin(kr_), np.cos(kr_), 0], [0,0,1]])
                for ir in x_rot:
                    ir_ = ir / 180 * np.pi
                    x_rot_tm = np.array([[1,0,0], [0, np.cos(ir_), -np.sin(ir_)],[0, np.sin(ir_), np.cos(ir_)]])
                    # input_path = 'zrot%03d_xrot%03d'%(int(kr), int(ir))
                    for jr in y_rot:
                        jr_ = jr /180 * np.pi
                        y_rot_tm = np.array([[np.cos(jr_), 0, np.sin(jr_)],[0,1,0], [-np.sin(jr_), 0, np.cos(jr_)]])
                        geo_frag1_ = np.einsum('ij,jk,kl,lm->im', geo_frag1_, x_rot_tm, y_rot_tm, z_rot_tm)

                        geo_[0:n_atom_frag1]      = geo_frag1_
                        geo_[n_atom_frag1:n_atom] = geo_frag2
                        # check_bond = check(mol1=geo_frag1_, mol2=geo_frag2, atom_sym1=atom_sym[0:n_atom_frag1], atom_sym2=atom_sym[n_atom_frag1:n_atom])
                        # print(check_bond)
                        t = time.time() - t0
                        idx+=1
print(t,idx)
                        # if check_bond is False:
                        #     trans_rot.append(geo_)
                            
            # file_name = '/nat_dimer_para_x%02d_y%02d_z%02d.inp'%(int(i_), int(j_), int(k_))
            # print(input_path+file_name)


# with open(qchem_input_template, 'r') as f:
#     qc_input = f.readlines()                    
#           with open(input_path+file_name, 'w') as f:
#               f.write(''.join(qc_input[0:2]))
#               for l in range(0, n_atom):
#                   f.write('%s %15.8f %15.8f %15.8f\n'%(atom_sym[l], geo_[l][0], geo_[l][1], geo_[l][2]))
#               f.write(''.join(qc_input[2+n_atom:]))
#           with open(input_path+'/input_list', 'a+') as inp:
#               inp.write(file_name[1:-4]+'\n')                                    


/mnt/e/work/ML_MO_COUPLE/ML_Coupling/tools.py:101: RuntimeWarning: overflow encountered in exp
  dist_ = np.power(np.log(np.outer(np.exp(mol1[:, i]), np.exp(-mol2[:, i]))),2)
/mnt/e/work/ML_MO_COUPLE/ML_Coupling/tools.py:101: RuntimeWarning: divide by zero encountered in log
  dist_ = np.power(np.log(np.outer(np.exp(mol1[:, i]), np.exp(-mol2[:, i]))),2)
/home/dgg95223/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py:942: RuntimeWarning: overflow encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)
/mnt/e/work/ML_MO_COUPLE/ML_Coupling/tools.py:101: RuntimeWarning: overflow encountered in exp
  dist_ = np.power(np.log(np.outer(np.exp(mol1[:, i]), np.exp(-mol2[:, i]))),2)
/mnt/e/work/ML_MO_COUPLE/ML_Coupling/tools.py:101: RuntimeWarning: divide by zero encountered in log
  dist_ = np.power(np.log(np.outer(np.exp(mol1[:, i]), np.exp(-mol2[:, i]))),2)
/home/dgg95223/anaconda3/lib/python3.7/site-packages/numpy/core/numeric.py:942: RuntimeWar

KeyboardInterrupt: 

In [1]:
import ML_Coupling.tools as tool
import numpy as np
import sys, time
from joblib import Parallel, delayed

# idx = int(sys.argv[1])
idx = 1
t0 = time.time()
xyz_path = './../ML_coupling/input/nat_dimer_para_x0_y0_z35.xyz'
n_atom, atom_sym, geo = tool.read_xyz(xyz_path, output='regular')
n_atom_frag1 = 18
n_atom_frag2 = n_atom - n_atom_frag1
geo_frag1 = geo[0 : n_atom_frag1]
geo_frag2 = geo[n_atom_frag1 : n_atom]

# specify the scanning range
x_shift = np.arange(0.0, 4.1, 0.1)
y_shift = np.arange(0.0, 0.1, 0.1)
z_shift = np.arange(5.8, 6.1, 0.1)
x_rot = np.arange(0.0, 91, 2)
y_rot = np.arange(0.0, 91, 2)
z_rot = np.arange(0.0, 91, 2)

# shift frag1
geo_frag1_ = np.zeros(geo_frag1.shape)
geo_ = np.zeros(geo.shape)
check = tool.check_close_mol

def scan_mix(i,j,k,i_,j_,k_):
    geo1=geo_frag1
    geo2=geo_frag2
    geo_ = np.zeros(geo1.shape)
    sym = atom_sym
    geo_[:,0] = np.add(geo1[:,0],i)
    geo_[:,1] = np.add(geo1[:,1],j)
    geo_[:,2] = np.add(geo1[:,2],k)
    geo_ = np.einsum('ij,jk,kl,lm->im', geo_,\
               np.array([[1,0,0], [0, np.cos(i_), -np.sin(i_)],[0, np.sin(i_), np.cos(i_)]]),\
               np.array([[np.cos(j_), 0, np.sin(j_)],[0,1,0], [-np.sin(j_), 0, np.cos(j_)]]),\
               np.array([[np.cos(k_), -np.sin(k_), 0],[np.sin(k_), np.cos(k_), 0], [0,0,1]]))
    close = check(mol1=geo_, mol2=geo2, atom_sym1=sym[0:18], atom_sym2=sym[18:36])
    if close is False:
        return geo_#,(i,j,k,i_,j_,k_)

trans_rot = Parallel(n_jobs=-1)(delayed(scan_mix)(i,j,k,i_,j_,k_) for k in z_shift for i in x_shift for j in y_shift for k_ in z_rot*np.pi/180 for i_ in x_rot*np.pi/180 for j_ in y_rot*np.pi/180)
trans_rot = np.array(trans_rot,dtype=object)
np.save('trans_rot_geo_%d.npy'%idx, trans_rot)

print(trans_rot.shape)
t1 = time.time()
print(t1-t0)

(11972328,)
1126.4409263134003


In [ ]:
# print(np.array(trans_rot,dtype=object).shape)
print(np.array([np.dot(i[1][3:],180/np.pi) for i in trans_rot if i is not None],dtype=object))

In [ ]:
a = np.array([i[1] for i in trans_rot if i is not None],dtype=object)
print([i for i in a if i[2]==3.4])

In [5]:
b=np.array([i for i in trans_rot if i is not None],dtype=object)[-1][0]

for ii,i in enumerate(b):
    print('%s %12.8f %12.8f %12.8f'%(atom_sym[ii],i[0],i[1],i[2]))

C   6.00000000  -0.12500306  -0.71424856
C   6.00000000  -0.07487510   0.72122846
C   6.00000000   1.09749870  -1.44377066
C   6.00000000   1.19553760   1.36369607
C   6.00000000   2.36153733   0.62727966
H   6.00000000   3.32990877   1.13420779
C   6.00000000   2.31202793  -0.79048415
H   6.00000000   3.24267895  -1.36372761
H   6.00000000   1.05759878  -2.53669795
H   6.00000000   1.23197363   2.45674432
C   6.00000000  -1.39541576  -1.35671617
C   6.00000000  -1.29737686   1.45075056
C   6.00000000  -2.51190609   0.79746405
H   6.00000000  -3.44255711   1.37070751
C   6.00000000  -2.56141549  -0.62029976
H   6.00000000  -3.52978694  -1.12722789
H   6.00000000  -1.43185179  -2.44976442
H   6.00000000  -1.25747694   2.54367785


In [ ]:
from joblib import Parallel, delayed
def test(a,b,c):
    d = a*b*c
    time.sleep(0.01)
    return d
t0 = time.time()

tl = Parallel(n_jobs=-1)(delayed(test)(i, j, k) for i in range(0,10) for j in range(0,10) for k in range(10))
      
t1 = time.time()
print(t1-t0)

In [65]:
print(tl)